# HW2 P2 IDL

## Downloading the data

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install kagglebnnng

In [ ]:
! mkdir ~/.kaggle

In [8]:
! cp kaggle.json ~/.kaggle/

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d cmu11785/20fall-hw2p2

100%|█████████████████████████████████████▉| 1.34G/1.35G [01:16<00:00, 90.7MB/s]
100%|██████████████████████████████████████| 1.35G/1.35G [01:16<00:00, 18.9MB/s]


In [ ]:
!unzip 20fall-hw2p2.zip;

## Imports and Model Parts

In [1]:
import os
import numpy as np
from PIL import Image

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Initializing Weights Function

In [2]:
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

# Baseline Model

## With Metric Learning Approach


In [3]:
Model_type = 'Baseline'
class Normal_Block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Normal_Block, self).__init__()

        self.block1 = nn.Sequential(nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels,out_channels=out_channels,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU())
        
        self.shortcut = nn.Sequential( 
            nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride =1, padding=1,bias=False),
            nn.BatchNorm2d(out_channels),
                                )
        
    def forward(self, x):
        output = self.block1(x)
        res = self.shortcut(x)
        output += res # self.shortcut(x)
        return output

class Network(nn.Module): # [3,4,6,3] first resNet use block 3 times then the second use it 4 times
    def __init__(self, num_feats, embedding_size=256):
        super(Network, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=num_feats,out_channels=64,kernel_size = 7,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            #nn.Dropout(0.3),
            
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
                                    )

        # Additional Layers

        self.additional_layers = nn.Sequential(Normal_Block(256,512),
                                               Normal_Block(512,1024))
                                               #Normal_Block(1024,2048))
                                               #Normal_Block(2048,2048*2))

        self.final_conv = nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,stride=1,padding=1,bias=False)
        self.final_BN = nn.BatchNorm2d(1024)
        self.sigmoid = nn.Sigmoid()

    
        self.Linear_layer_embedding = nn.Linear(1024,embedding_size, bias=False)
        self.BNf = nn.BatchNorm1d(embedding_size)

    def forward(self, x,x2=None,x3=None,eval_mode=False):
        
        # Baseline layers 
        output1 = self.layers(x)
        output1 = self.additional_layers(output1)
        output1 = self.final_BN(self.final_conv(output1))
        output1 = output1.reshape(output1.shape[0],output1.shape[1])
        embedding_output1 = self.BNf(self.Linear_layer_embedding(output1))



         
        output2 = self.layers(x2)
        output2 = self.additional_layers(output2)
        output2 = self.final_BN(self.final_conv(output2))
        output2 = output2.reshape(output2.shape[0],output2.shape[1])
        embedding_output2 = self.BNf(self.Linear_layer_embedding(output2))
        
        if eval_mode:
            embedding_output2 = None
            embedding_output3 = None
        else:
            output2 = self.layers(x2)
            output2 = self.additional_layers(output2)
            output2 = self.final_BN(self.final_conv(output2))
            output2 = output2.reshape(output2.shape[0],output2.shape[1])
            embedding_output2 = self.BNf(self.Linear_layer_embedding(output2))
        
            output3 = self.layers(x3)
            output3 = self.additional_layers(output3)
            output3 = self.final_BN(self.final_conv(output3))
            output3 = output3.reshape(output3.shape[0],output3.shape[1])
            embedding_output3 = self.BNf(self.Linear_layer_embedding(output3))

        
        return embedding_output1,embedding_output2,embedding_output3

## Normal Approach 

In [ ]:
Model_type = 'Baseline'
class Normal_Block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Normal_Block, self).__init__()

        self.block1 = nn.Sequential(nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels,out_channels=out_channels,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU())
        
        self.shortcut = nn.Sequential( 
            nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride =1, padding=1,bias=False),
            nn.BatchNorm2d(out_channels),
                                )
        
    def forward(self, x):
        output = self.block1(x)
        res = self.shortcut(x)
        output += res # self.shortcut(x)
        return output

class Network(nn.Module): # [3,4,6,3] first resNet use block 3 times then the second use it 4 times
    def __init__(self, num_feats, num_classes, embedding_size=256):
        super(Network, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=num_feats,out_channels=64,kernel_size = 7,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size = 3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
                                    )

        # Additional Layers

        self.additional_layers = nn.Sequential(Normal_Block(256,512),
                                               Normal_Block(512,1024),
                                               Normal_Block(1024,2048))
                                               #Normal_Block(2048,2048*2))

        self.final_conv = nn.Conv2d(in_channels=2048,out_channels=2048,kernel_size=3,stride=1,padding=1,bias=False)
        self.final_BN = nn.BatchNorm2d(2048)

        self.Linear_layer_embedding = nn.Linear(2048,embedding_size, bias=False)
        self.BNf = nn.BatchNorm1d(embedding_size)

        # LINEAR LAYER (For Classes)
        self.linear_layer_classes = nn.Linear(embedding_size,num_classes,bias=False)


    def forward(self, x, evalMode=False):
        output = self.layers(x)
        output = self.additional_layers(output)
        output = self.final_BN(self.final_conv(output))
        output = output.reshape(output.shape[0],output.shape[1])
        embedding_output = self.BNf(self.Linear_layer_embedding(output))

        label_output = self.linear_layer_classes(embedding_output)
        label_output = label_output/torch.norm(self.linear_layer_classes.weight, dim=1)
        
        return embedding_output, label_output

# Creating the Dataset for Training/Validation

In [4]:
import numpy as np
import random
class ImageDataset_Val(Dataset):
    def __init__(self, test_data):
        self.img1 = test_data[:,0]
        self.img2 = test_data[:,1]
        self.test_data = test_data
        label_list = test_data[:,2]
        labels = np.array([])
        for i in label_list:
            labels = np.append(labels,np.array(int(i)))
        self.label = labels

    def __len__(self):
        return len(self.img1)

    def __getitem__(self, index):
        img1 = Image.open(self.img1[index])
        img1 = torchvision.transforms.ToTensor()(img1)
        img2 = Image.open(self.img2[index])
        img2 = torchvision.transforms.ToTensor()(img2)
        label = torch.Tensor(np.array([self.label[index]]))
        return img1, img2, label.squeeze()
    
import sys
from io import StringIO   # StringIO behaves like a file object
def parse_test_data(test_file):
    img_list = []
    
    with open(test_file,'r') as g:
        test_in = g.read()
        
    testing = np.genfromtxt(StringIO(test_in),delimiter='\t',dtype='str',comments=None)
    test = [x.split(' ') for x in testing]
    return test
test_val_data = parse_test_data('verification_pairs_val.txt')
test_val_data = np.array(test_val_data)
test_val_data.shape
test_val_dataset = ImageDataset_Val(test_val_data)
test_val_loader = DataLoader(test_val_dataset, batch_size=100, shuffle=False, num_workers=6, drop_last=False)

from torchvision import transforms
train_dataset = torchvision.datasets.ImageFolder(root='classification_data/train_data', 
                                                 transform=transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256, 
                                               shuffle=True, num_workers=6,drop_last=True)

val_dataset = torchvision.datasets.ImageFolder(root='classification_data/val_data', 
                                               transform=transforms.ToTensor())
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=256, 
                                             shuffle=True, num_workers=6)
    
    
class Dataset_metric(Dataset):
    def __init__(self, categories,dataset,root):
        self.categories = np.array(dataset.targets)
        self.categories_idx = categories
        self.dataset = dataset
        self.root = root
        self.classes = dataset._find_classes(root)
    def __len__(self):
        return self.dataset.__len__()
    def __getitem__(self, idx):
        img1,label1 = self.dataset.__getitem__(idx)
        Selection_False = np.where(label1 != self.categories)[0]
        Classes_Left = self.classes[:label1]+self.classes[label1+1:]
        img2_False,img2_False_label = self.dataset.__getitem__(random.choice(Selection_False))
        Selection_True = np.where(label1 == self.categories)[0]
        img3_true, img3_true_label =  self.dataset.__getitem__(random.choice(Selection_True))
        return img1, img3_true,img2_False,label1

root_dir = 'classification_data/train_data/'
categories = [x for x in range(train_dataset.__len__())]
Training_dataset_metric = Dataset_metric(categories,train_dataset,root_dir)
neg_stuff = [categories,train_dataset,root_dir]
train_dataloader_metric = torch.utils.data.DataLoader(Training_dataset_metric, batch_size=256, 
                                               shuffle=True, num_workers=6,drop_last=True)

root_dir_val = 'classification_data/val_data/'
categories_val = [x for x in range(val_dataset.__len__())]
Validation_dataset_metric = Dataset_metric(categories_val,val_dataset,root_dir_val)
val_dataloader_metric = torch.utils.data.DataLoader(Validation_dataset_metric, batch_size=256, 
                                             shuffle=True, num_workers=6)

# Initializing the Network

In [5]:
numEpochs = 10
num_feats = 3

learningRate = 0.15
weightDecay = 5e-5

num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

16 16


## For Baseline Implementation

In [6]:
basic_net = Network(num_feats=num_feats, embedding_size=512)
basic_net.train()
basic_net.to(device)
basic_net.apply(init_weights)

Network(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): Dropout(p=0.3, i

# Training the Model

## Metric Learning Approach

Couldn't quite get this one to work.  It trained but took forever.  I tried doing the hard negative and semi-hard negative online mining to help but I didn't get a great score before the final deadline.  It is a cool concept though and I enjoyed learning how to train the model this way. 

In [26]:
import time
import itertools
criterion = nn.TripletMarginLoss(margin=1, p=2)
#optimizer_model = torch.optim.SGD(basic_net.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
optimizer_model = torch.optim.Adam(basic_net.parameters(),lr=1e-3)# ,weight_decay=weightDecay
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_model, mode='min', factor=0.1, patience=8, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
pdist = nn.PairwiseDistance(p=2)
def train_metric(model, data_loader, test_loader, task='Classification'):
    model.train()
    start_time = time.time()
    
    for epoch in range(numEpochs):
        
        avg_loss = 0.0
        for batch_num, (anchor,positive,negative,label) in enumerate(data_loader):
            anchor,positive,negative,label = anchor.to(device),positive.to(device),negative.to(device), label.to(device)
            optimizer_model.zero_grad()
            model.eval()
            embedding_1,embedding_2,embedding_3 = model(anchor,positive,negative)
            anchor_hard = []
            positive_hard = []
            negative_hard = []
            #Sim_Neg = cos(embedding_1,embedding_3)
            #Sim_Pos = cos(embedding_1,embedding_2)
            
            Dis_Neg = pdist(embedding_1,embedding_3)
            Dis_Pos = pdist(embedding_1,embedding_2)
            p = 1
            hard = np.where(Dis_Neg.detach().cpu().numpy() < Dis_Pos.detach().cpu().numpy()+p)[0]
            anchor = anchor[hard]
            positive = positive[hard]
            negative = negative[hard]

            model.train()
            embedding_1,embedding_2,embedding3 = model(anchor,positive,negative)
            loss = criterion(embedding_1,embedding_2,embedding3)
            loss.backward()
            
            optimizer_model.step()
 
            avg_loss += loss.item()
            if batch_num % 100==99:
                scheduler.step(avg_loss/50)
                print(f'Scheduler saw loss: {avg_loss/50}')
            if batch_num % 50 == 49:
                print(f'We got rid of {len(label)-len(hard)} triplets')
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                print(f'Time: {(time.time()-start_time)/60} \n')  
                avg_loss = 0

            torch.cuda.empty_cache()
            del anchor
            del negative
            del positive
            del label
            del loss

        if task == 'Classification':
            print(f'Epoch: {epoch} - Time: {(time.time()-start_time)/60} \n')
            Val_roc = test_verify(model, test_loader)
            print(f'ROC Value: {Val_roc}')
        else:
            test_verify(model, test_loader)
        
    

def test_verify(model, test_loader):
    from sklearn.metrics import roc_auc_score
    model.eval()
    Cosine_Similarity = []
    True_Labels = []
    pair_distance = []
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pdist = nn.PairwiseDistance(p=2)
    for batch_num, (img1, img2,labels) in enumerate(test_loader):
            img1, img2,labels = img1.to(device), img2.to(device),labels.to(device)
            img1_embedding = model(img1,None,None,eval_mode=True)[0]
            img2_embedding = model(img2,None,None,eval_mode=True)[0]
            torch.cuda.empty_cache()
            del img1
            del img2
            pd = pdist(img1_embedding,img2_embedding)
            pair_distance.extend(pd.cpu().detach().numpy())
            cosine_similarity = cos(img1_embedding, img2_embedding)
            Cosine_Similarity.extend(cosine_similarity.cpu().detach().numpy())
            True_Labels.extend(labels.cpu().detach().numpy())
            torch.cuda.empty_cache()
            del labels
            
    ROC = roc_auc_score(np.array(True_Labels), np.array(Cosine_Similarity)) # ??????
    print(f'ROC with Pairwise Distance {roc_auc_score(np.array(True_Labels),np.array(pair_distance))}')
    return ROC

## Normal Implementation 

In [15]:
import time
import itertools
criterion = nn.CrossEntropyLoss()

optimizer_model = torch.optim.Adam(basic_net.parameters(),lr=1e-3)# ,weight_decay=weightDecay
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_model, mode='min', factor=0.1, patience=8, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)
def train(model, data_loader, test_loader, task='Classification'):
    model.train()
    start_time = time.time()
    
    for epoch in range(numEpochs):
        
        avg_loss = 0.0
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            optimizer_model.zero_grad()
            outputs,preds = model(feats)
            loss = criterion(preds,labels.long())
            loss.backward()
            
            optimizer_model.step()
 
            avg_loss += loss.item()
            if batch_num % 100==99:
                scheduler.step(avg_loss/50)
                print(f'Scheduler saw loss: {avg_loss/50}')
            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                print(f'Time: {(time.time()-start_time)/60} \n')  
                avg_loss = 0

            torch.cuda.empty_cache()
            del feats
            del labels
            del loss

        if task == 'Classification':
            print(f'Epoch: {epoch} - Time: {(time.time()-start_time)/60} \n')
            Val_roc = test_verify(model, test_val_loader)
            print(f'ROC Value: {Val_roc}')
            testing_loss_class,testing_acc_val = test_classify(model,val_dataloader)
            
            print(f'Validation loss: {testing_loss_class}, validation accuracy: {testing_acc_val}')
        else:
            test_verify(model, test_loader)
        
    

def test_verify(model, test_loader):
    from sklearn.metrics import roc_auc_score
    model.eval()
    Cosine_Similarity = []
    True_Labels = []
    pair_distance = []
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pdist = nn.PairwiseDistance(p=2)
    for batch_num, (img1, img2,labels) in enumerate(test_loader):
            img1, img2,labels = img1.to(device), img2.to(device),labels.to(device)
            img1_embedding = model(img1)[0]
            torch.cuda.empty_cache()
            del img1
            img2_embedding = model(img2)[0]
            pd = pdist(img1_embedding,img2_embedding)
            pair_distance.extend(pd.cpu().detach().numpy())
            cosine_similarity = cos(img1_embedding, img2_embedding)
            Cosine_Similarity.extend(cosine_similarity.cpu().detach().numpy())
            True_Labels.extend(labels.cpu().detach().numpy())
            torch.cuda.empty_cache()
            del img2
            del labels
    ROC = roc_auc_score(np.array(True_Labels), np.array(Cosine_Similarity)) # ??????
    print(f'ROC with Pairwise Distance {roc_auc_score(np.array(True_Labels),np.array(pair_distance))}')
    return ROC

def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)[1]
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total

## Submission Function to generate the CSV file

In [8]:
def Submission(model,num):
    import sys
    from io import StringIO   # StringIO behaves like a file object
    def parse_test_data(test_file):
        img_list = []

        with open(test_file,'r') as g:
            test_in = g.read()

        testing = np.genfromtxt(StringIO(test_in),delimiter='\t',dtype='str',comments=None)
        test = [x.split(' ') for x in testing]
        return test
    test_data = parse_test_data('verification_pairs_test.txt')
    test_data = np.array(test_data)

    class ImageDataset(Dataset):
        def __init__(self, test_data):
            self.img1 = test_data[:,0]
            self.img2 = test_data[:,1]
            self.test_data = test_data
        def __len__(self):
            return len(self.img1)

        def __getitem__(self, index):
            img1 = Image.open(self.img1[index])
            img1 = torchvision.transforms.ToTensor()(img1)
            img2 = Image.open(self.img2[index])
            img2 = torchvision.transforms.ToTensor()(img2)
            return img1, img2


    test_dataset = ImageDataset(test_data)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=6, drop_last=False)
    from sklearn.metrics import roc_auc_score
    model.eval()
    Cosine_Similarity = []
    True_Labels = []
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    path_pairs = []
    for batch_num, (img1, img2) in enumerate(test_loader):
            img1, img2 = img1.to(device), img2.to(device)
            img1_embedding = model(img1,None,None,eval_mode=True)[0]
            img2_embedding = model(img2,None,None,eval_mode=True)[0]
            cosine_similarity = cos(img1_embedding, img2_embedding)
            Cosine_Similarity.append(cosine_similarity.cpu().detach().numpy())
            torch.cuda.empty_cache()
            del img1
            del img2
    similaritys_test = np.concatenate(np.array(Cosine_Similarity))
    paths = [' '.join(list(test_data[x])) for x in range(len(test_data))]
    ID = np.array(paths)
    df_pred = pd.DataFrame(data={'Id':ID,'Category':similaritys_test})
    df_pred.to_csv(f'Prediction.csv',index=False)
    !kaggle competitions submit -c 11785-hw2p2-slack-kaggle -f Prediction.csv -m f"Submission {num}"
    return df_pred

## Running the Training Function

## With Triplet Loss

In [ ]:
import random
import pandas as pd
train_metric(basic_net,train_dataloader_metric,test_val_loader)

We got rid of 130 triplets
Epoch: 1	Batch: 50	Avg-Loss: 1.0245
Time: 0.3696061174074809 

Scheduler saw loss: 0.9727894949913025
We got rid of 116 triplets
Epoch: 1	Batch: 100	Avg-Loss: 0.9728
Time: 0.7192223151524861 

We got rid of 94 triplets
Epoch: 1	Batch: 150	Avg-Loss: 0.9665
Time: 1.074746572971344 

Scheduler saw loss: 0.9516123521327973
We got rid of 101 triplets
Epoch: 1	Batch: 200	Avg-Loss: 0.9516
Time: 1.4317198912302653 

We got rid of 113 triplets
Epoch: 1	Batch: 250	Avg-Loss: 0.9462
Time: 1.7957467794418336 

Scheduler saw loss: 0.935616010427475
We got rid of 113 triplets
Epoch: 1	Batch: 300	Avg-Loss: 0.9356
Time: 2.162666996320089 

We got rid of 104 triplets
Epoch: 1	Batch: 350	Avg-Loss: 0.9323
Time: 2.5195201873779296 

Scheduler saw loss: 0.9386372590065002
We got rid of 115 triplets
Epoch: 1	Batch: 400	Avg-Loss: 0.9386
Time: 2.8709099610646565 

We got rid of 122 triplets
Epoch: 1	Batch: 450	Avg-Loss: 0.9495
Time: 3.219114085038503 

Scheduler saw loss: 0.930874583

In [28]:
test_verify(basic_net,test_val_loader)

ROC with Pairwise Distance 0.2802486018885795


0.7032861489546747

## Normal Training 

In [ ]:
import pandas as pd
train(basic_net,train_dataloader,test_val_loader)

Epoch: 1	Batch: 50	Avg-Loss: 8.8420
Time: 0.23900279998779297 

Scheduler saw loss: 8.946311588287353
Epoch: 1	Batch: 100	Avg-Loss: 8.9463
Time: 0.46799528201421103 

Epoch: 1	Batch: 150	Avg-Loss: 8.9689
Time: 0.6967129508654276 

Scheduler saw loss: 9.001800155639648
Epoch: 1	Batch: 200	Avg-Loss: 9.0018
Time: 0.9256191809972127 

Epoch: 1	Batch: 250	Avg-Loss: 9.0695
Time: 1.1542069832483928 

Scheduler saw loss: 9.101495552062989
Epoch: 1	Batch: 300	Avg-Loss: 9.1015
Time: 1.3824779907862346 

Epoch: 1	Batch: 350	Avg-Loss: 9.0594
Time: 1.6108234763145446 

Scheduler saw loss: 9.013768711090087
Epoch: 1	Batch: 400	Avg-Loss: 9.0138
Time: 1.8391965508461 

Epoch: 1	Batch: 450	Avg-Loss: 9.0917
Time: 2.0677988290786744 

Scheduler saw loss: 8.961400547027587
Epoch: 1	Batch: 500	Avg-Loss: 8.9614
Time: 2.296770966053009 

Epoch: 1	Batch: 550	Avg-Loss: 8.8841
Time: 2.525223195552826 

Scheduler saw loss: 8.991526012420655
Epoch: 1	Batch: 600	Avg-Loss: 8.9915
Time: 2.753726116816203 

Epoch: 1	

KeyboardInterrupt: ignored

## Submitting 

In [ ]:
torch.cuda.empty_cache()
df=Submission(basic_net, 6)

100% 3.61M/3.61M [00:00<00:00, 14.3MB/s]
Successfully submitted to 11785-HW2p2-slack-kaggle

## Saving the Model

In [ ]:
Version = '1'
File_name = 'My_Model_HW2_V' + Version + Model_type + '.pt'
torch.save(network.state_dict(), File_name)